## source_Extract_Field_class

In [1]:
# -*- coding: utf-8 -*-
from glob import glob
import nltk 
from nltk import tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import re
from time import time
from unicodedata import normalize

pd.set_option('display.max_colwidth',-1)

import json
import psycopg2
import pg
from datetime import datetime

## datos prueba

In [2]:

databese_info={
             'localhost':'localhost',
             'user':'postgres',
             'pass':'',
             'db':'pqr_electricaribe'
}

In [3]:
class TypeDataBase():
    def __init__(self,json_db):
        self.database = json_db
        self.cn = ''
        self.dic={'id':'','sspd':'','error':''}
        self.conect_database(json_db)
        self.update_flag(self.dic)
        
    def conect_database(self,json_db):
        try:
            self.cn = psycopg2.connect("dbname= "
                                  +json_db['db']+
                                  " user= "
                                  +json_db['user']+
                                  " host= "
                                  +json_db['localhost']+
                                  " password= "
                                  +json_db['pass']+" ")
        except psycopg2.DatabaseError as de:
            if self.cn:
                self.cn.rollback()
                print('Error %s'%de)
        return self.cn
    
    def serch_id(self,json):
        self.cn = self.conect_database(self.database)
        cur = self.cn.cursor()
        id_bu = ''
        try:
            cur.execute("select id from businesscase where serial='%s' AND botstate is null"%(json['sspd']))
            id_bu = cur.fetchall()
        except psycopg2.DatabaseError as de:
             json['error'] = 'Error %s'%de
        finally:
            if self.cn:
                self.cn.close()
        if id_bu:
                json['id'] = id_bu[0][0]
        else:
            json["id"] ='none'
                
        return json
    
    def update_flag(self,jso_n):
        self.cn = self.conect_database(self.database)
        cur = self.cn.cursor()
        try:
            cur.execute("update businesscase set botstate=%s where id=%s"%(1,jso_n['id']))
            self.cn.commit()
        except psycopg2.DatabaseError as de:
            jso_n['error'] = 'Error %s'%de
        finally:
            if self.cn:
                self.cn.close()
                
    def update_casoentescontrol(self,json):
        self.cn = self.conect_database(self.database)
        cur = self.cn.cursor()
        try:
            cur.execute("update casoentescontrol set documentoantecesor=%s,numerodecisionempresaeca=%s,"+
                        "numeroradicadoportico=%s where id=%s",(json['expediente_padre'],json['num_decision'], json['radication_RE'],json['id']))
            self.cn.commit()
            self.update_flag(json)
        except psycopg2.DatabaseError as de:
            json['error'] = 'Error %s'%de
        finally:
            if self.cn:
                self.cn.close()
        

## Class_Database

In [4]:
#from typeDataBase import TypeDataBase
objDataBase = TypeDataBase(databese_info)
field = {
      "date_decision": " 21 julio 2017 ", 
      "date_radication_re": "", 
      "expediente_padre": "20178201051872",
      'id':0,
      'error': '',
      "num_decision": "5116594", 
      "radication_RE": "", 
      "resolucion": "confirmar", 
      "sspd": "20178200393215", 
      "sspd_fecha": "01112017",
      "error": ''
}
id_json = objDataBase.serch_id(field)
#objDataBase.update_casoentescontrol(id_json)

id_json


{'date_decision': ' 21 julio 2017 ',
 'date_radication_re': '',
 'error': '',
 'expediente_padre': '20178201051872',
 'id': 'none',
 'num_decision': '5116594',
 'radication_RE': '',
 'resolucion': 'confirmar',
 'sspd': '20178200393215',
 'sspd_fecha': '01112017'}

In [5]:
type(field)
field is dict

False

## Class_Extract_Field

In [6]:
from extractField import extractField
obj_test = extractField('/home/jhonex/document_e1/',databese_info)
obj_test.organizeDocs()
obj_test.cleanDocs()
obj_test.extractFields()

Cleaning took 21.38 seconds
[{'sspd_fecha': '151121201', 'date_radication_re': '25 julio 2016', 'date_decision': ' 8 agosto 2016 ', 'id': 'none', 'resolucion': 'modificar', 'sspd': '20173200473935', 'expediente_padre': '', 'num_decision': '4164666', 'radication_RE': 're1180201627151', 'error': 'Error invalid input syntax for integer: "none"\nLINE 1: ...666\',numeroradicadoportico=\'re1180201627151\' where id=\'none\'\n                                                                 ^\n'}, {'sspd_fecha': '', 'date_radication_re': '05 julio 2017', 'date_decision': ' 16 junio 2017 ', 'id': 'none', 'resolucion': 'confirmar', 'sspd': '20178200393205', 'expediente_padre': '20178201055872', 'num_decision': '5020051', 'radication_RE': 're1110201723181', 'error': 'Error invalid input syntax for integer: "none"\nLINE 1: ...051\',numeroradicadoportico=\'re1110201723181\' where id=\'none\'\n                                                                 ^\n'}, {'sspd_fecha': '151122017', 'date_ra

[{'date_decision': ' 8 agosto 2016 ',
  'date_radication_re': '25 julio 2016',
  'error': 'Error invalid input syntax for integer: "none"\nLINE 1: ...666\',numeroradicadoportico=\'re1180201627151\' where id=\'none\'\n                                                                 ^\n',
  'expediente_padre': '',
  'id': 'none',
  'num_decision': '4164666',
  'radication_RE': 're1180201627151',
  'resolucion': 'modificar',
  'sspd': '20173200473935',
  'sspd_fecha': '151121201'},
 {'date_decision': ' 16 junio 2017 ',
  'date_radication_re': '05 julio 2017',
  'error': 'Error invalid input syntax for integer: "none"\nLINE 1: ...051\',numeroradicadoportico=\'re1110201723181\' where id=\'none\'\n                                                                 ^\n',
  'expediente_padre': '20178201055872',
  'id': 'none',
  'num_decision': '5020051',
  'radication_RE': 're1110201723181',
  'resolucion': 'confirmar',
  'sspd': '20178200393205',
  'sspd_fecha': ''},
 {'date_decision': ' ',
  '

### shooter Extractor field

In [7]:
flag = True
while flag:
    cn = conect_datbase(databese_info)
    cur = cn.cursor()
    try:
        cur.execute('select * from businesscase where botstate is null')
      
    except psycopg2.DatabaseError as de:
        print("don't worry, i continue->",de)
                
    finally:
        if cn:
            cn.close()
    
    

NameError: name 'conect_datbase' is not defined

In [ ]:
def update_casoentescontrol(self,json):
        try:
            cur.execute("update businesscase set description={0}, botstate={1} where id={2}".format(json['date_decision'],1,json['id']))
            self.cn.commit()
            id_bu = cur.fetchall()
            if id_bu:
                json['id'] = str(id_bu[0][0])
        except psycopg2.DatabaseError as de:
             json['error']= psycopg2.DatabaseError
    
    def update_casoentescontrol(self,json):
        self.cn = self.conect_datbase(self.database)
        cur = self.cn.cursor()
        try:
            cur.execute("UPDATE casoentescontrol SET  documentoantecesor=%s,numerodecisionempresaeca=%s, numeroradicadoportico=%s WHERE id=%s",
            (json['expediente_padre'],json['num_decision'],json['radication_RE'],json['id']))
            self.cn.commit()
        except psycopg2.DatabaseError as de:
             json['error']= psycopg2.DatabaseError               
        finally:
            if self.cn:
                self.cn.close()